In [2]:
import os
import pandas as pd
from data import GameManager

if not os.path.exists('data.xlsx'):
    import streamlit as st
    file_id = st.secrets["FILE_ID"]
    gm = GameManager(file_id)
    gm.download_data()

print("Data Available")
gm = GameManager()
df = gm.get_table_data()

Data Available
Reading Table: RESULTS_LOG


In [3]:
df.head()

,GAME_ID,DATE,SIDE,WONDERS,PLAYER,WONDER_POINTS,TREASURY_POINTS,MILITARY_POINTS,CIVILIAN_POINTS,COMMERCIAL_POINTS,SCIENCE_POINTS,GUILD_POINTS,TOTAL_POINTS,PLAYER_REF,RANK,WIN
0,1.0,2024-06-22,B,The Great Pyramid of Gizah,Eric,20.0,12.0,-1.0,6.0,22.0,0.0,0.0,59,Eric,1,True
1,1.0,2024-06-22,B,The Colossus of Rhodes,Sally,7.0,4.0,17.0,14.0,3.0,2.0,9.0,56,Sally,3,False
2,1.0,2024-06-22,B,The Mausoleum at Halicarnassus,Jenny,3.0,9.0,12.0,0.0,0.0,31.0,0.0,55,Jenny,4,False
3,1.0,2024-06-22,B,The Hanging Gardens of Babylon,Lance,0.0,0.0,2.0,14.0,0.0,26.0,9.0,51,Lance,6,False
4,1.0,2024-06-22,B,The Statue of Zeus at Olympia,Curry,5.0,3.0,11.0,19.0,11.0,0.0,10.0,59,Curry,2,False


# Player Analysis

In [5]:
def get_players_base_stats(df):
    players_df = df.groupby('PLAYER').agg({
        'GAME_ID': 'count',
        'WIN': 'sum',
        'TOTAL_POINTS': 'mean'
    }).reset_index()

    players_df = players_df.rename(columns={
        'GAME_ID': 'GAMES PLAYED',
        'TOTAL_POINTS': 'AVG POINTS'
    })

    players_df['WINNING RATE'] = players_df.apply(lambda x: x['WIN'] / x['GAMES PLAYED'],  axis=1)
    players_df = players_df.sort_values('WINNING RATE', ascending=False)

    return players_df

def get_player_wonder_stats(df):
    player_wonder_stats = df.groupby(['PLAYER', 'WONDERS']).agg({
        'GAME_ID': 'count',
        'RANK': 'mean',
        'WIN': 'sum'
    }).reset_index()

    player_wonder_stats = player_wonder_stats.rename(columns={
        'GAME_ID': 'GAMES PLAYED',
        'RANK': 'AVG RANK',
    })

    player_wonder_stats['WINNING RATE'] = player_wonder_stats.apply(lambda x: x['WIN'] / x['GAMES PLAYED'], axis=1)

    return player_wonder_stats

def get_player_best_performing_wonder(player_wonder_stats, columns_to_keep=[]):
    best_performing = player_wonder_stats.loc[player_wonder_stats.groupby('PLAYER')['WINNING RATE'].idxmax()]
    best_performing = best_performing.rename(columns={
        'WONDERS': 'BEST WONDER',
        'GAMES PLAYED': 'BEST WONDER GAMES',
        'AVG RANK': 'BEST WONDER AVG RANK',
        'WINNING RATE': 'BEST WONDER WIN RATE'
    })

    if not columns_to_keep:
        columns_to_keep = best_performing.columns
    
    return best_performing[columns_to_keep]

def join_best_performing_wonder_with_players():
    players_df = get_players_base_stats(df)

    player_wonder_stats = get_player_wonder_stats(df)
    best_performing = get_player_best_performing_wonder(player_wonder_stats, ['PLAYER', 'BEST WONDER', 'BEST WONDER WIN RATE'])

    players_df = pd.merge(players_df, best_performing, on='PLAYER')

    return players_df

join_best_performing_wonder_with_players()

,PLAYER,GAMES PLAYED,WIN,AVG POINTS,WINNING RATE,BEST WONDER,BEST WONDER WIN RATE
0,Jenny,16,5,56.750000,0.312500,The Mausoleum at Halicarnassus,0.666667
1,Curry,14,3,54.142857,0.214286,The Mausoleum at Halicarnassus,1.000000
2,Shasha,5,1,51.600000,0.200000,The Hanging Gardens of Babylon,0.500000
3,Lance,16,3,54.000000,0.187500,The Statue of Zeus at Olympia,0.500000
4,Sally,16,2,53.812500,0.125000,The Hanging Gardens of Babylon,0.333333
5,Yin,14,1,51.500000,0.071429,The Hanging Gardens of Babylon,0.333333
6,Eric,16,1,49.937500,0.062500,The Great Pyramid of Gizah,0.500000
7,Jerry,2,0,47.000000,0.000000,The Colossus of Rhodes,0.000000


In [17]:
player_wonder_stats = df.groupby(['PLAYER', 'WONDERS']).agg({
    'GAME_ID': 'count',
    'RANK': 'mean',
    'WON': 'sum'
}).reset_index()

player_wonder_stats.columns = ['PLAYER', 'WONDER', 'GAMES_PLAYED', 'AVG_RANK', 'WINS']
player_wonder_stats['WIN_RATE'] = player_wonder_stats['WINS'] / player_wonder_stats['GAMES_PLAYED']
player_wonder_stats

,PLAYER,WONDER,GAMES_PLAYED,AVG_RANK,WINS,WIN_RATE
0,Curry,The Colossus of Rhodes,3,3.333333,1,0.333333
1,Curry,The Great Pyramid of Gizah,2,4.000000,0,0.000000
2,Curry,The Hanging Gardens of Babylon,3,5.666667,0,0.000000
3,Curry,The Mausoleum at Halicarnassus,2,1.000000,2,1.000000
4,Curry,The Statue of Zeus at Olympia,3,3.333333,0,0.000000
5,Curry,The Temple of Artemis at Ephesus,1,7.000000,0,0.000000
6,Eric,The Colossus of Rhodes,1,6.000000,0,0.000000
7,Eric,The Great Pyramid of Gizah,2,3.500000,1,0.500000
8,Eric,The Hanging Gardens of Babylon,2,5.000000,0,0.000000
9,Eric,The Lighthouse of Alexandria,1,5.000000,0,0.000000
